### Plotting for validation after Seurat v3 batch effect correction
Author: Juliska E Boer

Date: 03 Nov 2020

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv
import scanpy.external as sce
import bbknn
import mnnpy as mnn
import scgen
import matplotlib.pyplot as plt
import matplotlib as mpl

#settings
sc.settings.verbosity = 3             # show: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.figdir = "../../figures/merge_diencephalon/"
sc.settings.set_figure_params(dpi=80)

C:\Users\Juliska\Anaconda3\envs\UMCU\lib\site-packages\anndata\_core\anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex
Using TensorFlow backend.


scanpy==1.4.6 anndata==0.7.1 umap==0.3.10 numpy==1.18.1 scipy==1.4.1 pandas==1.0.1 scikit-learn==0.22.1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


In [2]:
c_cont_red = mpl.colors.LinearSegmentedColormap.from_list('custom_red', 
                                             [(0, 'silver'),
                                              (0.1, (1, 0.82, 0.82)),
                                              (0.2, (1, 0.67, 0.67)),
                                              (0.3, (1, 0.58, 0.58)),
                                              (0.4, (1, 0.48, 0.48)),
                                              (0.5, (1, 0.38, 0.38)),
                                              (0.6, (1, 0.29, 0.29)),
                                              (0.7, (1, 0.22, 0.22)),
                                              (0.8, (1, 0.15, 0.15)),
                                              (0.9, (0.67, 0.09, 0.09)),
                                              (1, (0.54, 0, 0))], N=128)
c_cont_green = mpl.colors.LinearSegmentedColormap.from_list('custom_green', 
                                             [(0, 'silver'),
                                              (0.1, (0.50, 0.93, 0.60)),
                                              (0.2, (0.4, 0.89, 0.5)),
                                              (0.3, (0.3, 0.81, 0.4)),
                                              (0.4, (0.2, 0.73, 0.3)),
                                              (0.5, (0.1, 0.69, 0.2)),
                                              (0.6, (0, 0.62, 0.185)),
                                              (0.7, (0, 0.55, 0.16)),
                                              (0.8, (0, 0.47, 0.15)),
                                              (0.9, (0, 0.36, 0.11)),
                                              (1, (0, 0.29, 0.06))], N=128)
c_cont_blue = mpl.colors.LinearSegmentedColormap.from_list('custom_blue', 
                                             [(0, 'silver'),
                                              (0.1, (0.67, 0.77, 1)),
                                              (0.2, (0.58, 0.68, 1)),
                                              (0.3, (0.48, 0.58, 1)),
                                              (0.4, (0.38, 0.48, 1)),
                                              (0.5, (0.29, 0.39, 1)),
                                              (0.6, (0.22, 0.32, 1)),
                                              (0.7, (0.15, 0.25, 1)),
                                              (0.8, (0.09, 0.19, 1)),
                                              (0.9, (0, 0.1, 0.75)),
                                              (1, (0, 0.1, 0.5))], N=128)

In [3]:
merged = sc.read_h5ad('../../data/output/merge_diencephalon/Diencephalon_MergedSeurat.h5ad')
merged

OSError: Unable to open file (unable to open file: name = '../../data/output/merge_diencephalon/Diencephalon_MergedSeurat.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
merged.obs["seurat_clusters"] = merged.obs["seurat_clusters"].astype(str)

In [ ]:
inbetween = ["Developmental Hb" if x=="Habenula" else x for x in merged.obs["orig.ident"]]
merged.obs["orig.ident"] = ["Diencephalon" if x=="diencephalon" else x for x in inbetween]
merged.uns["orig.ident_colors"] = ["#66c2a5", "#ac8751"]

In [ ]:
sub = merged.obs_vector('Pou4f1') > 0.25
sub = np.where(sub==True, "Pou4f1+", sub) 
sub = np.where(sub=='False', "neuron", sub) 
merged.obs["cell_type"] = sub
merged.uns["cell_type_colors"] = ["red", "lightgrey"]

In [ ]:
sc.pl.umap(merged, color=['cell_type', 'orig.ident', 'seurat_clusters'], wspace=0.5, save="_seurat.pdf")

In [ ]:
merged.obs["pou4f1_expr"] = merged.obs_vector('Pou4f1')

In [ ]:
sc.pl.umap(merged, color='pou4f1_expr', vmin=0, color_map = c_cont_red, save="_Pou4f1_seurat.pdf")